# Hospital Readmission Risk Model - Feature Engineering

**Project:** Hospital Readmission Risk Prediction  
**Timeline:** January 2015 - May 2015  
**Author:** Blake Sonnier  

## Objective
Transform raw, inconsistent EHR data into standardized features suitable for machine learning:
- Standardize inconsistent data formats
- Translate medical codes across different systems
- Create time-series features from admission history
- Handle missing values appropriately
- Engineer clinically meaningful features

**Key Challenge:** Working with real EHR data required creating a robust preprocessing pipeline that could handle multiple hospital systems with different data standards.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path to import our custom modules
sys.path.append('../src')

# Import our custom modules
from data_processing import EHRDataProcessor
from feature_engineering import (
    MedicalCodeTranslator, 
    TemporalFeatureEngineer, 
    ClinicalFeatureEngineer,
    ComprehensiveFeatureEngineer
)
from visualization import ReadmissionVisualizer

print("Feature engineering libraries loaded successfully")
print(f"Processing started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Load Processed Data from Previous Step

Starting with the cleaned data from the data exploration phase, or loading fresh data and processing it.

In [ ]:
# Load the raw data (in practice, this would be the output from 01_data_exploration.ipynb)
data_path = '../data/sample_data.csv'

print("Loading and processing raw EHR data...")
raw_data = pd.read_csv(data_path)
print(f"Loaded raw dataset: {raw_data.shape}")

# Initialize data processor
processor = EHRDataProcessor()

# Apply basic data processing
processed_data, processing_report = processor.process_pipeline(raw_data)

print(f"\nData processing completed:")
print(f"Original shape: {processing_report['original_shape']}")
print(f"Processed shape: {processing_report['final_shape']}")
print(f"Steps completed: {len(processing_report['steps_completed'])}")

# Display sample of processed data
print("\nSample of processed data:")
processed_data.head()

## 2. Medical Code Translation System

### Challenge Solution: Unified Medical Code Mapping
One of the biggest challenges was standardizing medical codes across different hospital systems. We created a comprehensive mapping system with clinical advisor input.

In [ ]:
# Initialize medical code translator
translator = MedicalCodeTranslator()

print("Medical Code Translation System")
print("=" * 40)
print(f"Monitoring {len(translator.condition_mappings)} clinical conditions")
print(f"Total code mappings: {len(translator.code_to_condition)}")

# Display condition mappings
print("\nConditions being tracked:")
for condition, mapping in translator.condition_mappings.items():
    total_codes = sum(len(codes) for codes in mapping.values() if isinstance(codes, list))
    print(f"  • {condition.title()}: {total_codes} code variations")

# Example of code translation
print("\n=== EXAMPLE CODE TRANSLATIONS ===")
sample_codes = [
    "250.00;401.9",              # ICD-9 codes
    "E11.9;I10;I50.9",           # ICD-10 codes  
    "DM2;HTN;HF",                # Local hospital codes
    "DIABETES;KIDNEY;HEART",     # Mixed local codes
    "250.01;I10;COPD"            # Mixed coding systems
]

for codes in sample_codes:
    translated = translator.translate_codes(codes)
    print(f"Input: {codes}")
    print(f"  → Conditions: {list(translated.keys())}")
    print()

In [ ]:
# Apply medical code translation to our dataset
print("Applying medical code translation to dataset...")

# Create condition features from diagnosis codes
data_with_conditions = translator.create_condition_features(processed_data, 'diagnosis_codes')

# Analyze condition prevalence
condition_summary = translator.get_condition_summary(processed_data, 'diagnosis_codes')

print(f"\nCondition Features Created:")
condition_cols = [col for col in data_with_conditions.columns if col.startswith('has_')]
print(f"Total condition features: {len(condition_cols)}")

print(f"\nCondition Prevalence in Dataset:")
for condition, stats in condition_summary.items():
    print(f"  • {stats['description']}: {stats['count']} patients ({stats['prevalence']*100:.1f}%)")

# Visualize condition prevalence
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
condition_prev = data_with_conditions[condition_cols].mean().sort_values(ascending=True)
condition_names = [col.replace('has_', '').replace('_', ' ').title() for col in condition_prev.index]
plt.barh(range(len(condition_prev)), condition_prev.values * 100)
plt.yticks(range(len(condition_prev)), condition_names)
plt.xlabel('Prevalence (%)')
plt.title('Medical Conditions Prevalence')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Comorbidity distribution
comorbidity_count = data_with_conditions[condition_cols].sum(axis=1)
plt.hist(comorbidity_count, bins=range(8), alpha=0.7, edgecolor='black')
plt.xlabel('Number of Comorbidities')
plt.ylabel('Number of Patients')
plt.title('Comorbidity Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAverage comorbidities per patient: {comorbidity_count.mean():.1f}")
print(f"Patients with multiple comorbidities (≥2): {(comorbidity_count >= 2).sum()} ({(comorbidity_count >= 2).mean()*100:.1f}%)")

## 3. Temporal Feature Engineering

### Creating time-series features from admission patterns
**Challenge**: Many patients had multiple admissions, requiring time-series analysis to extract meaningful patterns.

In [ ]:
# Initialize temporal feature engineer
temporal_engineer = TemporalFeatureEngineer()

print("=== TEMPORAL FEATURE ENGINEERING ===")
print("Creating time-series features from admission patterns...")

# For demonstration, we'll simulate temporal features since our sample data
# represents single admissions. In the real project, this would work with 
# actual admission history data.

# Simulate days since last admission based on previous admissions count
np.random.seed(42)
def simulate_days_since_last_admission(row):
    if row['previous_admissions'] == 0:
        return 999  # First-time patient
    else:
        # Simulate realistic gaps between admissions
        return max(1, int(np.random.exponential(90)))

data_with_conditions['days_since_last_admission'] = data_with_conditions.apply(
    simulate_days_since_last_admission, axis=1
)

# Create admission pattern features
temporal_data = temporal_engineer.create_admission_pattern_features(
    data_with_conditions, 
    'days_since_last_admission',
    'previous_admissions'
)

print(f"\nTemporal features created:")
temporal_features = ['days_since_last_admission', 'recent_admission', 
                    'frequent_readmitter', 'admission_pattern']
for feature in temporal_features:
    if feature in temporal_data.columns:
        print(f"  ✓ {feature}")

# Analyze temporal patterns
print(f"\n=== TEMPORAL PATTERN ANALYSIS ===")

# Recent admission analysis
recent_rate = temporal_data['recent_admission'].mean()
print(f"Recent admissions (≤30 days): {recent_rate*100:.1f}%")

# Frequent readmitter analysis
frequent_rate = temporal_data['frequent_readmitter'].mean()
print(f"Frequent readmitters: {frequent_rate*100:.1f}%")

# Admission pattern distribution
pattern_dist = temporal_data['admission_pattern'].value_counts()
print(f"\nAdmission pattern distribution:")
for pattern, count in pattern_dist.items():
    print(f"  • {pattern}: {count} ({count/len(temporal_data)*100:.1f}%)")

# Analyze relationship with readmissions
if 'readmission_30_day' in temporal_data.columns:
    print(f"\n=== TEMPORAL RISK ANALYSIS ===")
    
    # Recent admission effect
    recent_readmission = temporal_data.groupby('recent_admission')['readmission_30_day'].mean()
    print(f"Readmission rate by recent admission:")
    print(f"  • Standard: {recent_readmission[0]*100:.1f}%")
    print(f"  • Recent (≤30 days): {recent_readmission[1]*100:.1f}%")
    
    # Frequent readmitter effect  
    frequent_readmission = temporal_data.groupby('frequent_readmitter')['readmission_30_day'].mean()
    print(f"\nReadmission rate by frequent readmitter status:")
    print(f"  • Standard: {frequent_readmission[0]*100:.1f}%")
    print(f"  • Frequent readmitter: {frequent_readmission[1]*100:.1f}%")
    
    # Pattern-based analysis
    pattern_readmission = temporal_data.groupby('admission_pattern')['readmission_30_day'].mean().sort_values(ascending=False)
    print(f"\nReadmission rate by admission pattern:")
    for pattern, rate in pattern_readmission.items():
        print(f"  • {pattern}: {rate*100:.1f}%")

In [ ]:
# Visualize temporal patterns
plt.figure(figsize=(15, 10))

# Days since last admission distribution
plt.subplot(2, 3, 1)
days_subset = temporal_data[temporal_data['days_since_last_admission'] < 999]['days_since_last_admission']
plt.hist(days_subset, bins=30, alpha=0.7, edgecolor='black')
plt.xlabel('Days Since Last Admission')
plt.ylabel('Number of Patients')
plt.title('Time Between Admissions\n(Excluding First-Time Patients)')
plt.axvline(days_subset.mean(), color='red', linestyle='--', label=f'Mean: {days_subset.mean():.0f} days')
plt.legend()
plt.grid(True, alpha=0.3)

# Recent admission by readmission status
plt.subplot(2, 3, 2)
if 'readmission_30_day' in temporal_data.columns:
    recent_cross = pd.crosstab(temporal_data['recent_admission'], temporal_data['readmission_30_day'], normalize='index')
    recent_cross.plot(kind='bar', ax=plt.gca(), color=['lightblue', 'orange'])
    plt.xlabel('Recent Admission (≤30 days)')
    plt.ylabel('Proportion')
    plt.title('Recent Admission vs Readmission')
    plt.xticks([0, 1], ['No', 'Yes'], rotation=0)
    plt.legend(['No Readmission', 'Readmission'])

# Frequent readmitter analysis
plt.subplot(2, 3, 3)
frequent_counts = temporal_data['frequent_readmitter'].value_counts()
plt.pie(frequent_counts.values, labels=['Standard', 'Frequent Readmitter'], autopct='%1.1f%%')
plt.title('Frequent Readmitter Distribution')

# Admission pattern distribution
plt.subplot(2, 3, 4)
pattern_counts = temporal_data['admission_pattern'].value_counts()
plt.bar(range(len(pattern_counts)), pattern_counts.values)
plt.xticks(range(len(pattern_counts)), pattern_counts.index, rotation=45)
plt.ylabel('Number of Patients')
plt.title('Admission Pattern Distribution')
plt.grid(True, alpha=0.3)

# Risk by admission pattern
plt.subplot(2, 3, 5)
if 'readmission_30_day' in temporal_data.columns:
    pattern_risk = temporal_data.groupby('admission_pattern')['readmission_30_day'].mean() * 100
    plt.bar(range(len(pattern_risk)), pattern_risk.values)
    plt.xticks(range(len(pattern_risk)), pattern_risk.index, rotation=45)
    plt.ylabel('Readmission Rate (%)')
    plt.title('Readmission Risk by Pattern')
    plt.grid(True, alpha=0.3)

# Previous admissions vs readmission
plt.subplot(2, 3, 6)
if 'readmission_30_day' in temporal_data.columns:
    prev_adm_risk = temporal_data.groupby('previous_admissions')['readmission_30_day'].mean() * 100
    prev_adm_risk = prev_adm_risk.head(8)  # Show first 8 categories
    plt.bar(range(len(prev_adm_risk)), prev_adm_risk.values)
    plt.xticks(range(len(prev_adm_risk)), prev_adm_risk.index)
    plt.xlabel('Previous Admissions')
    plt.ylabel('Readmission Rate (%)')
    plt.title('Risk by Admission History')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Clinical Feature Engineering

### Creating features based on medical domain knowledge

In [ ]:
# Initialize clinical feature engineer
clinical_engineer = ClinicalFeatureEngineer()

print("=== CLINICAL FEATURE ENGINEERING ===")
print("Creating domain-specific clinical features...")

# Step 1: Age-related features
age_data = clinical_engineer.create_age_features(temporal_data, 'age')
print(f"✓ Age features created")

# Step 2: Comorbidity features
comorbidity_data = clinical_engineer.create_comorbidity_features(age_data)
print(f"✓ Comorbidity features created")

# Step 3: Risk stratification
risk_data = clinical_engineer.create_risk_stratification(comorbidity_data)
print(f"✓ Risk stratification features created")

# Step 4: Utilization features
final_clinical_data = clinical_engineer.create_utilization_features(risk_data)
print(f"✓ Utilization features created")

# Summary of clinical features created
clinical_features = [
    'age_group', 'elderly', 'emergency_elderly',
    'comorbidity_count', 'multiple_comorbidities', 'high_risk_comorbidity',
    'clinical_risk_score', 'risk_category', 'high_risk_patient',
    'los_category', 'long_stay', 'admission_history'
]

print(f"\nClinical features successfully created:")
for feature in clinical_features:
    if feature in final_clinical_data.columns:
        print(f"  ✓ {feature}")

print(f"\nClinical feature summary:")
print(f"  • Elderly patients (≥75): {final_clinical_data['elderly'].sum()} ({final_clinical_data['elderly'].mean()*100:.1f}%)")
print(f"  • Multiple comorbidities: {final_clinical_data['multiple_comorbidities'].sum()} ({final_clinical_data['multiple_comorbidities'].mean()*100:.1f}%)")
print(f"  • High-risk patients: {final_clinical_data['high_risk_patient'].sum()} ({final_clinical_data['high_risk_patient'].mean()*100:.1f}%)")
print(f"  • Long stay patients: {final_clinical_data['long_stay'].sum()} ({final_clinical_data['long_stay'].mean()*100:.1f}%)")

In [ ]:
# Analyze clinical risk stratification
plt.figure(figsize=(15, 10))

# Age group distribution
plt.subplot(2, 4, 1)
age_group_counts = final_clinical_data['age_group'].value_counts()
plt.pie(age_group_counts.values, labels=age_group_counts.index, autopct='%1.1f%%')
plt.title('Age Group Distribution')

# Clinical risk score distribution
plt.subplot(2, 4, 2)
plt.hist(final_clinical_data['clinical_risk_score'], bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Clinical Risk Score')
plt.ylabel('Number of Patients')
plt.title('Clinical Risk Score Distribution')
plt.axvline(final_clinical_data['clinical_risk_score'].mean(), color='red', linestyle='--')
plt.grid(True, alpha=0.3)

# Risk category distribution
plt.subplot(2, 4, 3)
risk_cat_counts = final_clinical_data['risk_category'].value_counts()
colors = ['green', 'yellow', 'orange', 'red']
plt.bar(range(len(risk_cat_counts)), risk_cat_counts.values, 
        color=colors[:len(risk_cat_counts)])
plt.xticks(range(len(risk_cat_counts)), risk_cat_counts.index, rotation=45)
plt.ylabel('Number of Patients')
plt.title('Risk Category Distribution')
plt.grid(True, alpha=0.3)

# Length of stay categories
plt.subplot(2, 4, 4)
los_cat_counts = final_clinical_data['los_category'].value_counts()
plt.bar(range(len(los_cat_counts)), los_cat_counts.values)
plt.xticks(range(len(los_cat_counts)), los_cat_counts.index)
plt.ylabel('Number of Patients')
plt.title('Length of Stay Categories')
plt.grid(True, alpha=0.3)

# Risk factors by readmission (if target available)
if 'readmission_30_day' in final_clinical_data.columns:
    # High-risk patient analysis
    plt.subplot(2, 4, 5)
    high_risk_readmission = final_clinical_data.groupby('high_risk_patient')['readmission_30_day'].mean() * 100
    plt.bar(['Standard Risk', 'High Risk'], high_risk_readmission.values, 
            color=['lightblue', 'red'], alpha=0.7)
    plt.ylabel('Readmission Rate (%)')
    plt.title('Risk Classification Performance')
    plt.grid(True, alpha=0.3)
    
    # Elderly patient analysis
    plt.subplot(2, 4, 6)
    elderly_readmission = final_clinical_data.groupby('elderly')['readmission_30_day'].mean() * 100
    plt.bar(['Non-Elderly', 'Elderly (≥75)'], elderly_readmission.values,
            color=['lightgreen', 'orange'], alpha=0.7)
    plt.ylabel('Readmission Rate (%)')
    plt.title('Age Effect on Readmission')
    plt.grid(True, alpha=0.3)
    
    # Multiple comorbidities analysis
    plt.subplot(2, 4, 7)
    comorbid_readmission = final_clinical_data.groupby('multiple_comorbidities')['readmission_30_day'].mean() * 100
    plt.bar(['Few Conditions', 'Multiple Comorbidities'], comorbid_readmission.values,
            color=['lightblue', 'purple'], alpha=0.7)
    plt.ylabel('Readmission Rate (%)')
    plt.title('Comorbidity Effect')
    plt.grid(True, alpha=0.3)
    
    # Risk score vs readmission
    plt.subplot(2, 4, 8)
    # Create risk score bins
    final_clinical_data['risk_score_bins'] = pd.cut(final_clinical_data['clinical_risk_score'], 
                                                   bins=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
    risk_bin_readmission = final_clinical_data.groupby('risk_score_bins')['readmission_30_day'].mean() * 100
    plt.bar(range(len(risk_bin_readmission)), risk_bin_readmission.values,
            color=plt.cm.Reds(np.linspace(0.3, 1, len(risk_bin_readmission))))
    plt.xticks(range(len(risk_bin_readmission)), risk_bin_readmission.index, rotation=45)
    plt.ylabel('Readmission Rate (%)')
    plt.title('Risk Score Calibration')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print clinical insights
if 'readmission_30_day' in final_clinical_data.columns:
    print(f"\n=== CLINICAL INSIGHTS ===")
    print(f"High-risk patients have {high_risk_readmission[1]/high_risk_readmission[0]:.1f}x higher readmission rate")
    print(f"Elderly patients have {elderly_readmission[1]/elderly_readmission[0]:.1f}x higher readmission rate")
    print(f"Multiple comorbidities increase risk by {comorbid_readmission[1]/comorbid_readmission[0]:.1f}x")